In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from datasetClass import KSLDataset
from LeNetClass import LeNet
from trainLoop import Trainer

# Загрузка данных

In [3]:
df = pd.read_csv(r"data/Train.csv")

In [4]:
df.groupby(by = "img_IDS").count().value_counts()

Label
1        6249
Name: count, dtype: int64

In [5]:
df

,img_IDS,Label
0,ImageID_33HMDMJ3,Temple
1,ImageID_V5POEX4O,Church
2,ImageID_89CCCCW6,Enough/Satisfied
3,ImageID_GNJXOWX5,Me
4,ImageID_7Q9LOP7R,Love
...,...,...
6244,ImageID_F2HTAA5P,Love
6245,ImageID_0D69ZQ4X,Church
6246,ImageID_E67IZXVJ,Seat
6247,ImageID_ENGL8NP3,You


In [6]:
df['Label'].value_counts()

Label
Enough/Satisfied    695
Seat                695
Mosque              695
Temple              694
Church              694
Love                694
Me                  694
You                 694
Friend              694
Name: count, dtype: int64

In [25]:
df["Label"] = df["Label"].astype("category")
df["target"] = df["Label"].cat.codes

In [26]:
df

,img_IDS,Label,target
0,ImageID_33HMDMJ3,Temple,7
1,ImageID_V5POEX4O,Church,0
2,ImageID_89CCCCW6,Enough/Satisfied,1
3,ImageID_GNJXOWX5,Me,4
4,ImageID_7Q9LOP7R,Love,3
...,...,...,...
6244,ImageID_F2HTAA5P,Love,3
6245,ImageID_0D69ZQ4X,Church,0
6246,ImageID_E67IZXVJ,Seat,6
6247,ImageID_ENGL8NP3,You,8


In [27]:
train_df, val_df = train_test_split(
    df,
    test_size=0.33, random_state=42, stratify=df["Label"])

# Загрузка изображений

In [28]:
images_dir = "data/Images/Images"

train_dataset = KSLDataset(train_df, images_dir)
val_dataset   = KSLDataset(val_df,   images_dir)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False, num_workers=4)

# Запуск обучающего цикла LeNet

In [57]:
model = LeNet(in_channels=3,
              num_classes=len(train_df.target.unique()), use_dropout=True)

trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=10)

Epoch 1/10 | train_loss: 2.2021 | val_loss: 2.1977 | val_ROC_AUC: 0.5575
Epoch 2/10 | train_loss: 2.1953 | val_loss: 2.1892 | val_ROC_AUC: 0.6246
Epoch 3/10 | train_loss: 2.1705 | val_loss: 2.1331 | val_ROC_AUC: 0.6421
Epoch 4/10 | train_loss: 2.0940 | val_loss: 2.0131 | val_ROC_AUC: 0.7050
Epoch 5/10 | train_loss: 1.9761 | val_loss: 1.8776 | val_ROC_AUC: 0.7463
Epoch 6/10 | train_loss: 1.8820 | val_loss: 1.8059 | val_ROC_AUC: 0.7741
Epoch 7/10 | train_loss: 1.7964 | val_loss: 1.7353 | val_ROC_AUC: 0.7950
Epoch 8/10 | train_loss: 1.7113 | val_loss: 1.6758 | val_ROC_AUC: 0.8122
Epoch 9/10 | train_loss: 1.6562 | val_loss: 1.6490 | val_ROC_AUC: 0.8177
Epoch 10/10 | train_loss: 1.5855 | val_loss: 1.6183 | val_ROC_AUC: 0.8224
Best val ROC AUC: 0.8224
